<a href="https://colab.research.google.com/github/fener95/tiffs-repo/blob/main/Gaza_s2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate(auth_mode= 'notebook',  #force =True,
                #scopes='https://www.googleapis.com/auth/earthengine'
                )
ee.Initialize(project='mapping-applications')
import geemap

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=TjJv7C9S8SvOYPFRCSoCTPxIVawx1AMKdXMhreZ5vvw&tc=-HYW9cogGwuptffTR8n9THECG2Nu0rQFv3SGX_7xCFQ&cc=ps5lRlfR0fHlapG5gzHoTc62qPoHAMAZK5r58dWquVI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AcvDMrBaJmP77fgOZCYOdxSxTCI6s5vFDdh5JEiYO03jINiQZ3pGn3jAlg4

Successfully saved authorization token.


In [ ]:
gaul = ee.FeatureCollection('FAO/GAUL/2015/level2')

In [ ]:
aoi = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Gaza Strip')).geometry()

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Preprocess funtions

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

"""
img23 = (
    s2
    .filterDate('2023-07-01', '2023-07-31')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
    .median()
    .clip(aoi)
    .reproject(crs='EPSG:4326', scale=10)
)

img24 = (
    s2
    .filterDate('2024-07-01', '2024-07-31')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
    .median()
    .clip(aoi)
    .reproject(crs='EPSG:4326', scale=10)
)
"""


# Calculate NDBI for the 2023 image
#ndbi23 = img23.normalizedDifference(['B12', 'B8']).rename('NDBI')

# Calculate NDBI for the 2024 image
#ndbi24 = img24.normalizedDifference(['B12', 'B8']).rename('NDBI')

#visualization = {'min': -1,'max': 1,'bands': ['NDBI']}

"\nimg23 = (\n    s2\n    .filterDate('2023-07-01', '2023-07-31')\n    # Pre-filter to get less cloudy granules.\n    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\n    .map(mask_s2_clouds)\n    .median()\n    .clip(aoi)\n    .reproject(crs='EPSG:4326', scale=10)\n)\n\nimg24 = (\n    s2\n    .filterDate('2024-07-01', '2024-07-31')\n    # Pre-filter to get less cloudy granules.\n    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\n    .map(mask_s2_clouds)\n    .median()\n    .clip(aoi)\n    .reproject(crs='EPSG:4326', scale=10)\n)\n"

In [ ]:
# split-map
"""
left_layer = geemap.ee_tile_layer(ndbi23, visualization, name="Gaza Sept23'")
right_layer = geemap.ee_tile_layer(ndbi24, visualization, name="Gaza Sept24'")
m.split_map(left_layer, right_layer)
m.centerObject(aoi, 14)
"""

'\nleft_layer = geemap.ee_tile_layer(ndbi23, visualization, name="Gaza Sept23\'")\nright_layer = geemap.ee_tile_layer(ndbi24, visualization, name="Gaza Sept24\'")\nm.split_map(left_layer, right_layer)\nm.centerObject(aoi, 14)\n'

# NDBI method

[Source:EEFA-book-A1.7-Humanitarian-applications](https://docs.google.com/document/d/1MIPIFMJakC6eNGOhlkXLcwjUKFyY6QJLHElzBCOfyn4/edit)

In [ ]:
# a func to add indices we are interested into

def addIndices(img):
    ndvi = img.normalizedDifference(['B8', 'B4']) \
    .rename('NDVI');
    ndbi = img.normalizedDifference(['B11', 'B8']) \
    .rename(['NDBI']);
    imgIndices = img.addBands(ndvi).addBands(ndbi)
    return imgIndices

In [ ]:
# median of 7 months 2023

img23 = (
    s2
    .filterDate('2023-01-01', '2023-08-06')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
    .map(addIndices)
    .median()
    .clip(aoi)
    #.reproject(crs='EPSG:4326', scale=10)
)

# one-day-23--------


#median 7 months of 2024

img24 = (
    s2
    .filterDate('2024-01-01', '2024-08-06')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
    .map(addIndices)
    .median()
    .clip(aoi)
    #.reproject(crs='EPSG:4326', scale=10)
)

#one-day-24-----



In [ ]:
#img24

## one-day-least-cloudy recent image

In [ ]:
def get_tile_ids(image_collection, aoi):
    """Get unique tile IDs in the filtered image collection."""
    return image_collection.filterBounds(aoi).aggregate_array('MGRS_TILE').distinct()

def get_least_cloudy_image_for_tile(tile_id, image_collection, aoi):
    """Get the least cloudy image for a given tile within the AOI."""
    filtered = image_collection.filterMetadata('MGRS_TILE', 'equals', tile_id)
    return filtered.sort('CLOUDY_PIXEL_PERCENTAGE').first().clip(aoi)

In [ ]:
# Function to process the image collection for a specific date range
def get_least_cloudy_image(date_start, date_end, aoi):
    # Filter the image collection by date, AOI, and cloud coverage
    collection = (s2.filterDate(date_start, date_end)
                      .filterBounds(aoi)
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                      .map(mask_s2_clouds)
                      .map(addIndices)
                      .sort('CLOUDY_PIXEL_PERCENTAGE')
                      .first()
                      .clip(aoi))
    return collection
# Get the least cloudy images for the specified date ranges
one_day_23 = get_least_cloudy_image('2023-07-01', '2023-08-05', aoi)
one_day_24 = get_least_cloudy_image('2024-07-01', '2024-08-05', aoi)

In [ ]:
one_day_23 # 4 july
one_day_24 # 3 july

## ViZ

In [ ]:
# Viz

# Define color palettes manually for ndvi
green_palette = ['#f7fcf5', '#e5f5e0', '#c7e9c0', '#a1d99b', '#74c476', '#41ab5d', '#238b45', '#006d2c', '#00441b']
pr_green_palette = ['#40004b', '#762a83', '#9970ab', '#c2a5cf', '#e7d4e8', '#f7f7f7', '#d9f0d3', '#a6dba0', '#5aae61', '#1b7837']
# Set-up "True color" visualization parameters.
TCImgVis = {
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1,
    'max': 0.3,
    'min': 0.0,
    'opacity': 1
}
#false color
FCImgVis = {
    'bands': ['B8', 'B4', 'B3'],
    'gamma': 1,
    'max': 0.3,
    'min': 0.0,
    'opacity': 1
}

## Mapping

In [ ]:
#m.clear_layers()
m = geemap.Map()
m.centerObject(aoi, 14)
m.addLayer(aoi, {}, 'AOI')

In [ ]:
# Display True-color one day
m.addLayer(one_day_23, TCImgVis,
'one day 23',0)
m.addLayer(one_day_24, TCImgVis,
'one day 24',0)

In [ ]:
# Display True-color composites.
m.addLayer(img23, TCImgVis,
'TC Pre-Event Median',0)
m.addLayer(img24, TCImgVis,
'TC Post-Event Median',0)

In [ ]:
# Display False-color composites.
m.addLayer(img23, FCImgVis,
'FC Pre-Event Median',0)
m.addLayer(img24, FCImgVis,
'FC Post-Event Median',0)

In [ ]:
# Display median NDVI composites.
m.addLayer(img23, {
    'min': 0,
    'max': 0.7,
    'bands': ['NDVI'],
    'palette': green_palette
}, 'NDVI Pre-Event Median',0)
m.addLayer(img24, {
    'min': 0,
    'max': 0.7,
    'bands': ['NDVI'],
    'palette': green_palette
}, 'NDVI Post-Event Median',0)

In [ ]:
# NDVi difference from july '23 and july '24

ndvi_diff = img24.select('NDVI').subtract(img23.select('NDVI'))
m.addLayer(ndvi_diff,
    {
      'min': -0.1,
      'max': 0.1,
      'palette': green_palette
    },
    'Difference Median NDVI',0);


In [ ]:
# Display median NDBI composites
m.addLayer(img23, {
    'min': -1,
    'max': 1,
    'bands': ['NDBI'],
    'palette': ['cyan', 'red']
}, 'NDBI Pre-Event Median',0)
m.addLayer(img24, {
    'min': -1,
    'max': 1,
    'bands': ['NDBI'],
    'palette': ['cyan', 'red']
}, 'NDBI Post-Event Median',0)

In [ ]:
m

Map(center=[31.40757091833705, 34.378018702181286], controls=(WidgetControl(options=['position', 'transparent_…

# Export to drive

In [ ]:
# mask the NoData values so they do not appear in the streamlit app as black rectangle inscribing the area:
image23 = one_day_23.updateMask(one_day_23.neq(0));
image24 = one_day_24.updateMask(one_day_24.neq(0));

In [ ]:
# here substitute img23 with the one_day_23

task_img23TC = ee.batch.Export.image.toDrive(
    image= image23.select(['B4', 'B3', 'B2']),      # here is the image with masked no data value. check it!
    description='gaza23_true_color_noData',
    folder='rasters',  # Folder in Google Drive
    fileNamePrefix='img23_true_color_noData',
    region=aoi,
    scale=10,
    crs='EPSG:4326',
    formatOptions={
        'cloudOptimized': True
    }
)

# here substitute img24 with the one_day_24

task_img24TC = ee.batch.Export.image.toDrive(
    image= image24.select(['B8', 'B4', 'B3']),
    description='gaza24_true_color_noData',
    folder='rasters',
    fileNamePrefix='img24_true_color_noData',
    region=aoi,
    scale=10,
    crs='EPSG:4326',
    formatOptions={
        'cloudOptimized': True
    }
)
"""
#---------------------------------------------------#
task_ndvi23 = ee.batch.Export.image.toDrive(
    image=img23.select('NDVI'),
    description='ndvi23',
    folder='GEE_Exports',
    fileNamePrefix='ndvi23',
    region=aoi,
    scale=10,
    crs='EPSG:4326',
    fileFormat='GeoTIFF'
)

task_ndvi24 = ee.batch.Export.image.toDrive(
    image=img24.select('NDVI'),
    description='ndvi24',
    folder='GEE_Exports',
    fileNamePrefix='ndvi24',
    region=aoi,
    scale=10,
    crs='EPSG:4326',
    fileFormat='GeoTIFF'
)
"""
#task_img23TC.start()
#task_img24TC.start()
#task_ndvi23.start()
#task_ndvi24.start()


In [ ]:
task_img23TC.status()

{'state': 'RUNNING',
 'description': 'gaza23_true_color_noData',
 'priority': 100,
 'creation_timestamp_ms': 1723544415191,
 'update_timestamp_ms': 1723544417853,
 'start_timestamp_ms': 1723544417818,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'D6VQHRZTWP2G3CTJBZUVAHRA',
 'name': 'projects/mapping-applications/operations/D6VQHRZTWP2G3CTJBZUVAHRA'}

In [ ]:
task_img24TC.status()

{'state': 'RUNNING',
 'description': 'gaza24_true_color_noData',
 'priority': 100,
 'creation_timestamp_ms': 1723544736398,
 'update_timestamp_ms': 1723544741871,
 'start_timestamp_ms': 1723544741835,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'OJW366DH6MCGTRY6T2K7I62M',
 'name': 'projects/mapping-applications/operations/OJW366DH6MCGTRY6T2K7I62M'}

In [ ]:
#
#task_ndvi23.status()
#task_ndvi24.status

# github url of tiffs to COG

In [ ]:

# URL of the TIFF file to be converted
url = 'https://github.com/fener95/tiffs-repo/raw/main/img23_true_color.tif'

# Path where the converted COG will be saved
out_cog = 'img23_true_color_cog.tif'

# Convert the TIFF file to a COG
geemap.image_to_cog(url, out_cog)

# Validate the newly created COG file
geemap.cog_validate(out_cog)
geemap.cog_validate(out_cog, verbose=True)